In [8]:
%matplotlib inline

import itertools
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import tensorflow
ps = PorterStemmer()

In [9]:
data=pd.read_csv('train.csv', encoding = "ISO-8859-1")
#test_data=pd.read_csv('test.csv', encoding = "ISO-8859-1")
data.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


In [98]:
def preProcessString(text):
    #Strips quotes at end of text
    text=text.strip('')
    #Rmoving twitter handles @user
    text=re.sub("@[\w]*"," ",text) 
    # Remove URLs with the space
    text = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' ', text)
    # Strip space, " and ' from text
    text = text.strip(' "\'')
    #remove digits
    #text = re.sub(r'(\d)', '', text)
    #Remove repeated letters of string such as jusssssst to just 
    text=re.sub(r'(.)\1{3,}', r'\1', text)
    # remove all special characters
    text = re.sub('[^A-Za-z]', ' ', text)
    #replace two or more dots with space
    text = re.sub("\\.{2,}"," ",text);
    # converting all text into small letters and store them as words for further processing
    text_list = text.lower().split()
     # stemming the words (removing prefix and postfix) using Porter stemming algorithm..
    text_list = [ps.stem(word) for word in text_list]
    return ' '.join(text_list)

In [99]:
data['Preprocessed_data']=data['SentimentText'].apply(preProcessString)

In [83]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split
train, test = train_test_split(data, test_size=0.20)
Count_vectorization=TfidfVectorizer(use_idf=True, max_features=20000)

In [84]:
x_train = train["Preprocessed_data"]
y_train = train["Sentiment"]
x_test  = test["Preprocessed_data"]
y_test  = test["Sentiment"]

In [106]:
from __future__ import print_function
from tensorflow import keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

# set parameters:
max_features = 20000
maxlen = 200
batch_size = 128
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 5



print('Loading data...')
x_train = train["Preprocessed_data"]
y_train = train["Sentiment"]
x_test  = test["Preprocessed_data"]
y_test  = test["Sentiment"]

x_train = [one_hot(d, max_features) for d in x_train]
y_train = np.asarray(y_train)
x_test  = [one_hot(d, max_features) for d in x_test]
y_test  = np.asarray(y_test)

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test,  maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(layers.Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(layers.Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(layers.Conv1D(filters, kernel_size, padding='valid', activation='relu', strides=1))
# we use max pooling:
model.add(layers.GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(layers.Dense(hidden_dims))
model.add(layers.Dropout(0.2))
model.add(layers.Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=batch_size, validation_split=0.2, epochs=epochs)

Loading data...
Pad sequences (samples x time)
x_train shape: (79991, 200)
x_test shape: (19998, 200)
Build model...
Train on 63992 samples, validate on 15999 samples
Epoch 1/5
63992/63992 [==============================] - 35s 554us/sample - loss: 0.5366 - accuracy: 0.7204 - val_loss: 0.4780 - val_accuracy: 0.7669
Epoch 2/5
63992/63992 [==============================] - 35s 544us/sample - loss: 0.4445 - accuracy: 0.7919 - val_loss: 0.4714 - val_accuracy: 0.7758
Epoch 3/5
63992/63992 [==============================] - 35s 543us/sample - loss: 0.3785 - accuracy: 0.8303 - val_loss: 0.4972 - val_accuracy: 0.7587
Epoch 4/5
63992/63992 [==============================] - 35s 543us/sample - loss: 0.3143 - accuracy: 0.8655 - val_loss: 0.5455 - val_accuracy: 0.7610
Epoch 5/5
63992/63992 [==============================] - 35s 544us/sample - loss: 0.2542 - accuracy: 0.8959 - val_loss: 0.5976 - val_accuracy: 0.7575


In [109]:
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size )
print(score)
print(acc)

19998/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

0.6022902415184774
0.7559256
